In [1]:
##################统计随机森林模型中最重要的特征##############  


import numpy as np
import pickle
import logging
np.random.seed(1337)
import re
import codecs
import jieba
import math
import numpy as np
from itertools import product, count  
from heapq import nlargest  
from gensim.models import word2vec  
from sklearn.utils import shuffle
try:
    reload(sys)
    sys.setdefaultencoding('utf-8')
except:
    pass

## 打开正面和负面的新闻
格式是 {标题：新闻内容}

In [2]:
import json
# with open("pos.JSON", 'r') as fp:
#     pos_text = json.load( fp)
# with open("neg.JSON", 'r') as fp:
#     neg_text = json.load( fp)
    
with open("/Users/daichanglin/Desktop/data/pos0307.JSON", 'r') as fp:
    pos_text = json.load( fp)
with open("/Users/daichanglin/Desktop/data/neg0307.JSON", 'r') as fp:
    neg_text = json.load( fp)
     

In [3]:
print(len(pos_text))
print(len(neg_text))

negfile = "/Users/daichanglin/Desktop/igoldenbeta/robotSVN/robot/robot-parent/python/data/negfactor.txt"
i=0
with open(negfile,'r',encoding = 'utf-8') as sw:
    for line in sw:
        neg_text["neg"+str(i)]=line
        i=i+1
        
print(len(pos_text))
print(len(neg_text))

2318
348
2318
2568


In [4]:
#分句
def sentence_split(str_centence):
    list_ret = list()
    for s_str in str_centence.split('。'):
        if '?' in s_str:
            list_ret.extend(s_str.split('？'))
        elif '!' in s_str:
            list_ret.extend(s_str.split('！'))
        else:
            list_ret.append(s_str.strip())
    return list_ret

In [5]:
#建立停词
stopwordset = set()
with open('sa/jieba_dict/stopwords.txt','r',encoding = 'utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

#分词wrapper
def seg_art_list(art_list):
    corpus = []
    for title, art in art_list.items():
        corpus.append(" ".join(seg_sent(art)))
    return corpus

#分词，去标点符号，去停词，去英文，去数字
#art： 单篇文章
#return：所有的单词
output = open('word_seg.txt','w')
def seg_sent(art):
    r = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'  
    art = art.strip()
    art = re.sub(r, '', art)
    art = re.sub("[a-z]","",art)
    art = re.sub("[0-9]","",art)
    l = []

    seg_list = list(jieba.cut(art, cut_all = False))
    for word in seg_list:
        if word not in stopwordset and word != ' ' and word != "\n" and word != "\n\n":
                output.write(word + ' ')
                l.append(word)
    return l


In [6]:
#建立和保存正面新闻的 分词
#建立和保存负面新闻的 分词

pos_tfidf = seg_art_list(pos_text)
neg_tfidf = seg_art_list(neg_text)
with open("pos_tfidf.JSON", 'w') as fp:
    json.dump(pos_tfidf, fp)
with open("neg_tfidf.JSON", 'w') as fp:
    json.dump(neg_tfidf, fp)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/nr/4rg65wr168q0jp4wjc2kkh100000gn/T/jieba.cache
Loading model cost 1.313 seconds.
Prefix dict has been built succesfully.


## 建模

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics.scorer import make_scorer
from sklearn import linear_model
from sklearn import metrics
import operator
from time import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import train_test_split as tts
import sklearn
import timeit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
#Y： 1为正面，-1为负面
X = list(pos_tfidf) + list(neg_tfidf)
y = [1] * len(pos_tfidf) + [-1]*len(neg_tfidf)

In [9]:
len(y)

4886

In [10]:

cv = CountVectorizer(ngram_range=(1,2), min_df=20,max_df = 0.9,lowercase=False)
_X = cv.fit_transform(X).toarray()
tfidf = TfidfVectorizer()
_X = tfidf.fit_transform(X)
feature_names = list(tfidf.get_feature_names())
classifier= RandomForestClassifier(n_estimators=100,random_state=5,min_samples_leaf=2)



In [11]:
X_train, X_test, y_train, y_test = tts(_X, y, test_size=0.2)

model = classifier.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("classifier result:")
print(clsr(y_test, y_pred, target_names=['neg', 'pos']))

print("confusion matrix")
print(metrics.confusion_matrix(y_test, y_pred))

outpath='model0607withFactor'
with open(outpath, 'wb') as f:
    print("dump model")
    pickle.dump(model, f)

print("Model written out to {}".format(outpath))

classifier result:
             precision    recall  f1-score   support

        neg       1.00      0.90      0.94       525
        pos       0.89      1.00      0.94       453

avg / total       0.95      0.94      0.94       978

confusion matrix
[[470  55]
 [  2 451]]
dump model
Model written out to model0607withFactor


In [12]:
feature_importance = pd.DataFrame({'name':feature_names, 'importance':model.feature_importances_},columns = ['name','importance'])

In [13]:
# feature_importance = pd.DataFrame({'name':feature_names, 'importance':model.feature_importances_},columns = ['name','importance'])
sort_importance = feature_importance.sort_values(by= ['importance'],ascending = False)
sort_importance.head(50)

,name,importance
31199,预计,0.024807
4178,产品,0.020913
16703,提升,0.017469
15565,我们,0.017123
1140,pe,0.016348
31179,预期,0.016313
22967,目前,0.015756
6268,公告,0.014764
26941,行业,0.014371
3478,主要,0.014001


In [14]:
sort_importance.to_csv("sort_importance.csv",index=False)